In [1]:
import pandas as pd
import joblib
import numpy as np
import scipy.sparse as sparse
import random
import implicit
from sklearn.preprocessing import MinMaxScaler

### Load Dataset

In [11]:
users_df = pd.read_csv("v1/data/users.csv")
items_df = pd.read_csv("v1/data/items.csv")
user_item_interaction_df = pd.read_csv("v1/data/user_item_interactions.csv")

### 1. Lightfm

In [12]:
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.data import Dataset
from lightfm import LightFM

In [13]:
items_df['brand'] = items_df['brand'].astype('category')
items_df['brand'] = items_df['brand'].cat.codes
items_df['category'] = items_df['category'].astype('category')
items_df['category'] = items_df['category'].cat.codes
items_features = [(row['price'], row['brand'], row['category']) for index, row in items_df.iterrows()]
items_df.head()

,id,title,price,brand,category
0,14521,"Oxy 10, 10gm",286.20,321,119
1,12352,Make-Up Studio Brush Cleanser(Ph0900),1785.00,264,57
2,11507,Buddsbuddy Silicone Orthodontic Pacifier (1pc),169.15,57,328
3,12401,Make-Up Studio Cream Blusher,2200.00,264,42
4,14408,Set Wet Hair Gel Cool- 50 Ml,104.00,381,158


In [14]:
users_df['gender'] = users_df['gender'].astype('category')
users_df['gender'] = users_df['gender'].cat.codes
user_features =  [(row['age'], row['gender']) for index, row in users_df.iterrows()]

In [15]:
dataset = Dataset()
dataset.fit(users_df['id'].values, items_df['id'].values,user_features=user_features, item_features=items_features)

In [ ]:
interactions, weights = dataset.build_interactions((row['user'], row['item'], row['rating']) for index, row in user_item_interaction_df.iterrows())



In [17]:
train, test = random_train_test_split(interactions)

In [18]:
lightfm_model = LightFM(learning_rate=0.05, loss='warp')
lightfm_model.fit(interactions, epochs=10)

In [40]:
joblib.dump(lightfm_model,"api/prediction/lightfm/model.joblib")
joblib.dump(dataset, "api/prediction/lightfm/id_mapper.joblib")

['api/prediction/lightfm/id_mapper.joblib']

In [19]:
train_precision = precision_at_k(lightfm_model, train, k=10).mean()
test_precision = precision_at_k(lightfm_model, test, k=10).mean()

train_auc = auc_score(lightfm_model, train).mean()
test_auc = auc_score(lightfm_model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.09, test 0.04.
AUC: train 0.95, test 0.95.


In [39]:
def get_similar_product(model, item_id):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors
    
    # Normalize the vectors to unit length
    tag_embeddings = (model.item_embeddings.T
                      / np.linalg.norm(model.item_embeddings, axis=1)).T
    
    query_embedding = tag_embeddings[item_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:10]
    
    return most_similar

print(items_df[(items_df['id']==14408)])

similar_products = get_similar_product(lightfm_model,14408)
for product in similar_products:
    print(items_df['title'][product])

      id                         title  price  brand  category
4  14408  Set Wet Hair Gel Cool- 50 Ml  104.0    381       158
Zoryl 3mg Tab
Axillary Crutches Small Size
Siptin 100mg Tab
Rotahaler
Farmasi Vfx Pro Camera Ready Foundation
Cosmo Body Spray Allemagne- 120 Ml
Zebita Xt Tab
Mi Simple Way Foaming Handwash
Farlin Anti-Mosquito Patch 002- Bck-002


In [ ]:
scores = lightfm_model.predict(0, np.arange(12000))
len(np.argsort(-scores))

### 2. Using Implicit

In [20]:
dataset_df = user_item_interaction_df.copy()

In [51]:
dataset_transformed = dataset_df.copy()
dataset_transformed['user'] = dataset_transformed['user'].astype("category")
dataset_transformed['user_id'] = dataset_transformed['user'].cat.codes

sparse_item_user = sparse.csr_matrix((dataset_transformed['rating'].astype(float), (dataset_transformed['item'], dataset_transformed['user_id'])))
sparse_user_item = sparse.csr_matrix((dataset_transformed['rating'].astype(float), (dataset_transformed['user_id'], dataset_transformed['item'])))

In [22]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
model.fit(interactions)

  0%|          | 0/50 [00:00<?, ?it/s]

In [95]:
ids, scores = model.recommend(1001, interactions.tocsr()[1001], N=10)
print(ids)

[9 8 7 6 5 4 3 2 1 0]


In [36]:
from implicit.evaluation import ranking_metrics_at_k
metrics = ranking_metrics_at_k(model, train, test, K=10, num_threads=4)
metrics

  0%|          | 0/14758 [00:00<?, ?it/s]

{'precision': 0.10577496814978471,
 'map': 0.0806387600721724,
 'ndcg': 0.12058011957293072,
 'auc': 0.5695938936430563}

In [96]:
joblib.dump(model, "api/prediction/implicit/model.joblib")
joblib.dump(interactions, "api/prediction/implicit/interactions.joblib")

['api/prediction/implicit/interactions.joblib']

### Implicit Using Different Interaction Set

In [99]:
implicit_model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
alpha = 15
data = (interactions * alpha).astype('double')
implicit_model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

In [100]:
from implicit.evaluation import ranking_metrics_at_k
metrics = ranking_metrics_at_k(implicit_model, train, test, K=10, num_threads=4)
metrics

  0%|          | 0/14758 [00:00<?, ?it/s]

{'precision': 0.10471060652485929,
 'map': 0.09267600858134414,
 'ndcg': 0.13455264767943448,
 'auc': 0.590765542578471}

In [104]:
def train_test_split_data(interactions, split_count, fraction=None):
    """
    Perform a temporal user split.
    
    Params
    ------
    interactions : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their last
        interactions into test set. If None, then all 
        users are considered.
        
    returns: train, test csr matrices.
    """
    k = 5
    # Note: likely not the fastest way to do things below.
    train = interactions.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
    
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except: 
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        print(interactions.getrow(user).indices)
        test_ratings = np.random.choice(interactions.getrow(user).indices, 
                                        size=split_count, 
                                        replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = interactions[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr()

In [106]:
train_1, test_1 = train_test_split_data(interactions, 1)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[13656 12782  5998  1411]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14753 14628 14558 14445 13893 13783 13651 13445 12699 12401 12348 12250
 12240 11482 11328 11318 11032 10677 10377  9152  8700  8681  8456  7990
  7373  6832  6240  5731  5663  5612  3617  3379  3365  3341  2731  2542
  2115  1615   321    18]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[12433  9327  4274    98]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[11434]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14153 13259 12798  4962  4401   939   189]
[14701 14534 14127 14124 14071 13990 13986 13973 13714 13357 13330 12157
 12002 11403 10166  9297  9004  8055  7743  7556  7155  7111  7105  7100
  6985  6762  6319  6113  5929  5447  5412  5235  5002  4962  4940  4871
  4350  4223  4052  3984  3974  3356  2865  2622  2538  2328  2171  2030
  1963  1729  1704   963   787   744   639   592   243   130]
[]
[]
[]
[]
[]
[]
[]
[5081 4216]
[]
[]
[]
[]
[]
[]
[]


[1625]
[]
[]
[14438 14177 14163 14155 13868 13771 13729 13660 13388 12798 12590 10855
  9748  7825  7556  7391  6985  6931  6012  5898  5412  4588  4047  2049
  1963  1738   744   171    19]
[13986 13166]
[13827]
[14037 13679 13495  6319   327]
[14447 13973 13256 10966 10932 10447  8108  7984  7556  7105  6632  5339
  5002  4944  3974  3766  3650  3485  2622  1920   592   183]
[]
[]
[12002 10826]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14736 14680 14645 14638 14516 14384 14260 14233 14163 14119 14115 14070
 14060 14047 13973 13964 13956 13925 13894 13892 13824 13776 13694 13661
 13630 13445 13226 13116 12888 12832 12160 12145 12048 11798 11640 11566
 11508 11192 11158 11018 10848 10771 10642 10575 10520 10151 10134  9658
  9412  9247  8714  8108  7764  7334  7191  7068  7037  6950  6795  6638
  6612  6464  6367  6034  6012  5769  5550  5485  5419  5274  5235  5225
  5224  5067  5066  5004  4942  4934  4907  4896  4894  4745  4588  4583
  4580  3967  3913  3775  3678  3531  3500  3485  3482  

[]
[]
[]
[]
[]
[]
[]
[]
[]
[13640]
[]
[]
[]
[]
[]
[10010  7026]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14157]
[]
[]
[13067]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[10912  8788  8339]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[13656]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[3650 1794  946  323]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[8257]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[13737 11203  7140  7040  6790  3638   172   121]
[]
[]
[3896]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[12292  9954  9394  7394  3785  1151]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14628 13597 12456 11467 11449 11152 10400 10226  8681  7954  4088    32]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14165]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[10295  6871]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[10966  4588]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[10733 10251]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[10150]
[]
[]
[]
[]
[]
[]
[]
[13113 12842 10809  8764  8501  6726  6320  5381  4511  4088  3705  2726
  1657   572]
[]
[]
[]
[]
[]
[]
[]
[14444 13827 13234 11861 11107  9441  9208  7972  7141  7077  7041  5503
  4805  3902  3775]
[]
[14061 13714 13660  5986  5916  5801  5788  5776  5730  5683  5682  5240
  5224]
[]
[]
[]
[]
[]
[]
[]
[]
[9236]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[11951  6034]
[]
[]
[]
[14691 11997 11856 10449  8882  8790  8636  7981]
[14584 14534 14447 14246 14225 14155 14089 13769 13742 13692 13525 13488
 13295 12647 12534 12525 12505 12134 11831 10966 10522 10260  9903  9840
  9664  8675  8510  7071  6703  5883  5129  4588  3708  2785  2303  2025
  2006  1662  1383  1310  1297   868   467   269   262   186]
[]
[]
[]
[]
[]
[

[]
[14669 14651 14534 14155 13911 13796 12563 12034  8322  7439  7279  7206
  6985  6319  3341  2542]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[339]
[14627 10732  1641]
[]
[10954  3371]
[14393 11717  9261  5602]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[14737]
[2008]
[]
[]
[]
[]
[7923]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[10517  9868  7944  6746]
[]
[]
[]
[]
[]
[]
[]
[]
[13986 13736  5389]
[14727 14671 14667 14613 14473 14432 14216 14177 14115 13851 13838 13783
 13463 13347 12851 12695 12638 12563 12403 12346 12153 12097 11993 11944
 11846 11811 11766 11628 11575 11499 11395 11328 11229 10881 10548 10412
 10387 10246 10226  9957  9676  9505  9484  9410  9192  9179  9051  8622
  8501  8463  8351  8262  8253  8162  8088  8060  8043  8033  8032  7764
  7742  7488  7478  7410  7401  7391  7247  7091  7078  7042  6970  6910
  6818  6611  6298  6218  6138  5609  5197  4941  4628  4560  4108  4095
  3934  39

...

[]
[14100 14061  2553]
[]
[5058]
[12002  4962]
[12789 11797  9354  6871  6611  5516  3603    32]
[3842]
[13740 13459 13435]
[14638 13736 13459 13435 13346  6146  5821  1729  1620  1548]
[14447 13359  9373  4588]
[14155 14061 13893 12589 12348  8747  7360  6802  4349  3541]
[14255 14164 13922 12767  4773  4450  2921  1069  1043  1013]
[12731  4854]
[1419]
[]
[14104  9836]
[14742 13289 13141 12699  8681  3212  1306   384]
[]
[10892  5235]
[13655 10136  9938  4940  4577]
[11467  8738  5516    32]
[9404 5332 2669]
[]
[8225 6432 3651 1105]
[14076 13782 11688 11594  9995  9594  7928  7495  6861  6709  6616  6088
  5975  4671  4572  4405  3603  3399  3212  3193  1337  1054   984   917]
[14051]
[]
[14019  9816  5911  3993]
[]
[13834 10226  7744  5235  4574  4331    32]
[14534 10937 10364  9988  6484  4588  4286  2773  1174]
[]
[]
[11194  3650]
[]
[9836 6744 6616  160  109   71]
[12888 12590]
[12829  3212]
[14558 11110 10347  8144  7343  6334  5468  4278  4270  3987  3942  3470
  1731   480   3

[14736  4052  1942]
[]
[12471]
[]
[]
[14600 14389 13771 12825 11381  5286  4910]
[14128 10003  7880   189]
[6554]
[13895 13390  6327  3739  2676  2669]
[14330 13409 12934 12840 12038 11969 11861  7078  5547  5309  5157  4886
  4158  3180  2499  2253  2109  1830   993   339   175]
[9624]
[2829 2106 1738]
[]
[14655 14470 14260 13996 13659 12945 12837  7895  7843  7744  7611  6532
  6205  6012  5412  5256  5250  5235  5203  5037  5004  4940  4907  4823
  4805  4729  4583  4469  4261  4231  3840  2538  2499  2004  1548  1387
   402   100]
[5235]
[14670 14447 14432 14255 14037 13827 13801 13747 13720 13459 13357 13226
 11163 11060 10593 10588  9658  9182  8731  8525  7556  7420  7345  7317
  7247  7209  6930  6767  6724  6282  6190  6027  5910  5705  5590  5523
  5514  5419  5397  5353  5337  5257  5081  5002  4829  4813  4802  4588
  4268  4223  4130  3531  3523  3513  3180  2954  2941  2915  2864  2826
  2763  2568  2512  2493  2326  2185  1920  1832  1829  1641  1514  1417
  1220  1123  

[263 115]
[14140 11640  5274  3408  2533]
[14736  7382  4582  4580]
[14497  4993  4882  4823  4745  4158  2341  2067  2017  1690]
[8078 6623 5962 4270 3988 3240 1582]
[14736 11469  6554  4942]
[11105  9398  7949  7804  7603  7561  6574  3647]
[]
[9474]
[14534 14478 14096 14037 13827 13768 13459 13445 13435 12468 12030 11839
 10932 10883 10845 10447 10151  7683  7131  7105  7086  7078  7032  6930
  6827  6319  5977  5416  5155  5002  4971  4588  4223  3729  2450  2157
  1970  1963   836]
[14233 13357  7611  7556  6862  5412  4910  2622]
[1719]
[12814 10620  9866  7867  3717  2008]
[]
[14273 13955 13860  7923  4384  2351]
[]
[10624  7014  4962  4940  3761  1427]
[13895 12464  6835  6327  4803  3622  1842  1811   109]
[5769]
[14306 13273  5235]
[11688 11641 11480 11408 10165  8480  7628  6697  4955  4531  4045  3353
  3306  3215  3019  1603  1539   418]
[14434 13460 12923 12768 11806  7441  1087]
[]
[6578 6070   77]
[]
[13650 13166 12584 12428 11348 10983 10624  7148  7139  6663  5800  36

[224]
[11837  8163  5147]
[7714]
[14714  5235]
[]
[11884  9989  3799  2791  1811  1484]
[13037  7986]
[10916  6818  6187  4459  3708]
[9476]
[14540 14096 13854 13260 12248  7895  6328  6285  5847  5576  4729  3830
  3749  3236  3216  3184  2793  2773  2478  1832  1801   777   729   672
   425   284    78    67]
[4159 3487 3429]
[11449 10150  9700  7132  3161]
[12253  7990  1723]
[7139 5233 2958]
[10891]
[11688  7935  5182  4566  4045  3414  3340  1691   480]
[14393 13854  7649  6012  5514  5259  4962  4588  2605  2069   554]
[7093]
[14742 13141]
[]
[14242 13477 12087 11828  7105  3598    91]
[3896]
[]
[7871 7029 4960 4588 3897 3377  958  939]
[14302 13887 13296 12174 10826 10739  7018  5752  3896  3352  2280]
[14672 14216 11448 11084 10225 10045 10041  9818  9311  9130  8796  8513
  7706  7439  4656  4074  4009  3717  3638    99]
[13660 13357 12888  4910  4588  2941  2773  2622  1963  1387   592]
[13284 12441 11294 11214 10618  9851  7962  7493  7193  6835  3212  1753
  1544  1513   86

TypeError: cannot unpack non-iterable NoneType object